In [1]:
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from utils import *

DATA_PATH = '../../data/esp/'

%load_ext autoreload
%autoreload 2

In [2]:
df = load_data(DATA_PATH,'train',nclasses=3).loc[:,['review_content','review_rate']]
df.head(5)

Dataset cargado para 3 clases (malo=0, medio=1, bueno=2)
Num samples per category:
0    184750
1     92375
2    184750
Name: review_rate, dtype: int64


,review_content,review_rate
0,"Medio berreta, no justifica el gasto, ya se sa...",0
1,Quizás a unas personas les sirve y a otras no ...,1
2,Excelente !! superó mi expectativas. Lo único ...,2
3,Es una buena relación calidad/precio. La cámar...,2
4,"Muy poca voluntad para ayudarme con cambiarlo,...",0


In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('../old/02 bias/bert/')
tokenizer

PreTrainedTokenizer(name_or_path='../old/02 bias/bert/', vocab_size=31002, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [6]:
from pprint import pprint

pprint(tokenizer('Queremos tokenizar esta frase'))
pprint(tokenizer('Queremos [MASK] esta frase'))

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'input_ids': [4, 5806, 1166, 981, 1014, 6596, 1359, 9712, 5],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [4, 5806, 0, 1359, 9712, 5],
 'token_type_ids': [0, 0, 0, 0, 0, 0]}


In [9]:
corpus = ['Tenemos una serie de frases',
          'y queremos tokenizarlas']
pprint(tokenizer(corpus))

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[4, 2339, 1108, 3100, 1008, 19672, 5],
               [4, 1042, 5806, 1166, 981, 1014, 6596, 1372, 5]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [13]:
encoded_inputs = tokenizer(df['review_content'].tolist(),padding=True)
pprint({key:val[:20] for key, val in encoded_inputs.items()})

{'attention_mask': [[1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1],
                    [1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
                     1,
               

In [17]:
encoded_inputs['inputs_ids'][0]

KeyboardInterrupt: 

In [15]:
max_len = 512
numdocs = len(df['review_content'])
attention_masks = np.zeros((numdocs,max_len),dtype=float)
indices = np.zeros((numdocs,max_len),dtype=int)
for i,sent in enumerate(df['review_content']):
    tokens = tokenizer.tokenize("[CLS] {} [SEP]".format(sent))
    numtokens = len(tokens)
    indices[i,:numtokens] = [tokenizer.convert_tokens_to_ids(tk) for tk in tokens]
    attention_masks[i,:numtokens] = 1.
encoded_input2 = {'input_ids': indices, 'attention_mask': attention_masks}
pprint({key:val[:20] for key, val in encoded_input2.items()})

ValueError: cannot copy sequence with size 666 to array axis with dimension 512

In [16]:
df['review_content'].str.len().max()

1526